# Capstone Project

This project will model the immigration data for the US in April of 2016, making it easily queriable for analysts. 


### 1 - Data 

- I94 Immigration: [Origin](https://travel.trade.gov/research/reports/i94/historical/2016.html)
- US City Demographics: [Origin](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)
- Airport Codes: [Origin](https://datahub.io/core/airport-codes#data)

The immigration data for US, it includes details of immigrants such as age groups and mode of transformation.
The demographics table gives the details about the city for a given city code. 
The airports table shows information about a particular airport.

#### Import the necessary libraries

In [110]:
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *


# display all columns in dataframe
pd.set_option('display.max_columns', None)
output_path='data/'

### 2 - Load the data

In [2]:
spark = SparkSession.builder/
                    .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")/
                    .enableHiveSupport()/
                    .getOrCreate()

immi_df = spark.read.csv('immigration_data_sample.csv', sep=',', header=True, inferSchema=True)
cit_df = spark.read.csv('us-cities-demographics.csv', sep=';', header=True, inferSchema=True)
air_df = spark.read.csv('airport-codes_csv.csv', sep=',', header=True, inferSchema=True)

##### Explore imigration data

In [61]:
immi_df.toPandas().describe(include='all')

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
count,1.000000e+03,1.000000e+03,1000.0,1000.0,1000.000000,1000.00000,1000,1000.000000,1000.000000,941,951.000000,1000.000000,1000.000000,1000.0,1.000000e+03,382,4,1000,954,0.0,954,1000.000000,1000,859,35.000000,967,1.000000e+03,992,1000
unique,NaN,NaN,NaN,NaN,NaN,NaN,70,NaN,NaN,51,NaN,NaN,NaN,NaN,NaN,97,3,9,10,0.0,1,NaN,99,3,NaN,101,NaN,502,10
top,NaN,NaN,NaN,NaN,NaN,NaN,NYC,NaN,NaN,FL,NaN,NaN,NaN,NaN,NaN,MEX,STU,G,O,NaN,M,NaN,07152016,M,NaN,AA,NaN,LAND,WT
freq,NaN,NaN,NaN,NaN,NaN,NaN,155,NaN,NaN,188,NaN,NaN,NaN,NaN,NaN,28,2,757,800,NaN,954,NaN,27,471,NaN,109,NaN,19,443
mean,1.542097e+06,3.040461e+06,2016.0,4.0,302.928000,298.26200,NaN,20559.680000,1.078000,NaN,20575.037855,42.382000,1.859000,1.0,2.016042e+07,NaN,NaN,NaN,NaN,NaN,NaN,1973.618000,NaN,NaN,3826.857143,NaN,6.937237e+10,NaN,NaN
std,9.152879e+05,1.799818e+06,0.0,0.0,206.485285,202.12039,NaN,8.995027,0.485955,NaN,24.211234,17.903424,0.386353,0.0,4.951657e+01,NaN,NaN,NaN,NaN,NaN,NaN,17.903424,NaN,NaN,221.742583,NaN,2.338134e+10,NaN,NaN
min,1.092500e+04,1.320800e+04,2016.0,4.0,103.000000,103.00000,NaN,20545.000000,1.000000,NaN,20547.000000,1.000000,1.000000,1.0,2.016040e+07,NaN,NaN,NaN,NaN,NaN,NaN,1923.000000,NaN,NaN,3468.000000,NaN,0.000000e+00,NaN,NaN
25%,7.214422e+05,1.412170e+06,2016.0,4.0,135.000000,131.00000,NaN,20552.000000,1.000000,NaN,20561.000000,30.750000,2.000000,1.0,2.016041e+07,NaN,NaN,NaN,NaN,NaN,NaN,1961.000000,NaN,NaN,3668.000000,NaN,5.599301e+10,NaN,NaN
50%,1.494568e+06,2.941176e+06,2016.0,4.0,213.000000,213.00000,NaN,20560.000000,1.000000,NaN,20570.000000,42.000000,2.000000,1.0,2.016042e+07,NaN,NaN,NaN,NaN,NaN,NaN,1974.000000,NaN,NaN,3887.000000,NaN,5.931477e+10,NaN,NaN
75%,2.360901e+06,4.694151e+06,2016.0,4.0,438.000000,438.00000,NaN,20567.250000,1.000000,NaN,20580.000000,55.000000,2.000000,1.0,2.016042e+07,NaN,NaN,NaN,NaN,NaN,NaN,1985.250000,NaN,NaN,3943.000000,NaN,9.343623e+10,NaN,NaN


##### Explore city data

In [60]:
cit_df.toPandas().describe(include='all')

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
count,2891,2891,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2891,2891,2.891000e+03
unique,567,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,5,NaN
top,Springfield,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA,Hispanic or Latino,NaN
freq,15,676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,676,596,NaN
mean,NaN,NaN,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,NaN,NaN,4.896377e+04
std,NaN,NaN,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,NaN,NaN,1.443856e+05
min,NaN,NaN,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,NaN,NaN,9.800000e+01
25%,NaN,NaN,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,NaN,NaN,3.435000e+03
50%,NaN,NaN,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,NaN,NaN,1.378000e+04
75%,NaN,NaN,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,NaN,NaN,5.444700e+04


##### Explore airport data

In [62]:
air_df.toPandas().describe(include='all')

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
count,55075,55075,55075,48069.000000,55075,55075,55075,49399,41030,9189,28686,55075
unique,55075,7,52144,NaN,7,244,2810,27133,40850,9042,27436,54874
top,SCDI,small_airport,Centre Hospitalier Heliport,NaN,NA,US,US-TX,Seoul,MBAC,0,LAN,"0, 0"
freq,1,33965,85,NaN,27719,22757,2277,404,3,80,5,53
mean,NaN,NaN,NaN,1240.789677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,1602.363459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,-1266.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,205.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,718.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,1497.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cleaning 

##### Only accept data that has i94addr in city table and i94port in airport table

In [84]:
immigration_df = imi_df.join(cit_df, immi_df['i94addr']==cit_df['State Code'], how='leftsemi') \
                       .join(air_df, immi_df['i94port']==air_df['local_code'], how='leftsemi') 

##### Drop records from the city table with null State Code

In [85]:
city_df = cit_df.dropna(how='any', subset=['State Code'])

##### Drop records from airport table with null local_code

In [86]:
airport_df = air_df.dropna(how='any', subset=['local_code'])

### 3 - Data Model
##### Fact table:

- <b>Immigrations</b>:
    - <b>cicid</b>: identifier
    - <b>year</b>: year 
    - <b>month</b>: month
    - <b>city</b>: code of city of origin
    - <b>resid</b>: residential code
    - <b>port</b>: port code
    - <b>arrivdate</b>: arrival date
    - <b>mode</b>: travel mode code
    - <b>addr</b>: state code
    - <b>depdate</b>: departure date
    - <b>age</b>: age
    - <b>visa</b>: visa code
    - <b>visapost</b>: post that issued visa
    - <b>occup</b>: occupation
    - <b>entdepa</b>: arrival flag
    - <b>entdepd</b>: departure flag
    - <b>entdepu</b>: update flag
    - <b>matflag</b>: match flag
    - <b>biryear</b>: year of birth
    - <b>dtaddto</b>: date to which admitted to U.S.
    - <b>gender</b>: gender
    - <b>insnum</b>: insurance number
    - <b>airline</b>: airline used
    - <b>admnum</b>: admission number
    - <b>fltno</b>: flight number
    - <b>visatype</b>: type of visa

##### Dimension tables:

- <b> Airport </b>:
    - <b>ident</b>: identification
    - <b>type</b>: airport type
    - <b>name</b>: airport name
    - <b>elevation_ft</b>: elevation
    - <b>continent</b>: continent
    - <b>iso_country</b>: iso country
    - <b>iso_region</b>: iso region
    - <b>municipality</b>: municipality
    - <b>gps_code</b>: gps code
    - <b>iata_code</b>: iata code
    - <b>local_code</b>: local code
    - <b>coordinates</b>: coordinates

- <b> City </b>:
    - <b>city</b>: City
    - <b>state</b>: State
    - <b>median_age</b>: Median Age
    - <b>male_population</b>: Male Population
    - <b>female_population</b>: Female Population
    - <b>total_population</b>: Total Population
    - <b>number_of_veterans</b>: Number of Veterans
    - <b>foreign_born</b>: Foreign-born
    - <b>average_household_size</b>: Average Household Size
    - <b>state_code</b>: State Code
    - <b>race</b>: Race
    - <b>count</b>: Count

### 4 - Run Pipelines to Model the Data 

In [107]:
get_date = F.udf(lambda x: (datetime(1960, 1, 1).date() + timedelta(x)).isoformat() if x is not None else '1960-01-01')

immigration = immigration_df.select(
                                    F.col("cicid").cast(IntegerType()),
                                    F.col("i94yr").cast(IntegerType()).alias('year'),
                                    F.col("i94mon").cast(IntegerType()).alias('month'),
                                    F.col("i94cit").cast(IntegerType()).alias('city'),
                                    F.col("i94res").cast(IntegerType()).alias('resid'),
                                    F.col("i94port").cast(StringType()).alias('port'),
                                    get_date(F.col("arrdate")).alias('arrivdate'),
                                    F.col("i94mode").cast(IntegerType()).alias('mode'),
                                    F.col("i94addr").cast(StringType()).alias('addr'),
                                    get_date(F.col("depdate")).alias('depdate'),
                                    F.col("i94bir").cast(IntegerType()).alias('age'),
                                    F.col("i94visa").cast(IntegerType()).alias('visa'),
                                    F.col("visapost").cast(StringType()).alias('visapost'),
                                    F.col("occup").cast(StringType()),
                                    F.col("entdepa").cast(StringType()),
                                    F.col("entdepd").cast(StringType()),
                                    F.col("entdepu").cast(StringType()),
                                    F.col("matflag").cast(StringType()),
                                    F.col("biryear").cast(IntegerType()),
                                    F.col("dtaddto").cast(StringType()),
                                    F.col("gender").cast(StringType()),
                                    F.col("insnum").cast(IntegerType()),
                                    F.col("airline").cast(StringType()),
                                    F.col("admnum").cast(FloatType()),
                                    F.col("fltno").cast(IntegerType()),
                                    F.col("visatype").cast(StringType()) 
                                   )

airport = airport_df.select(
                            F.col('ident').cast(StringType()),
                            F.col('type').cast(StringType()),
                            F.col('name').cast(StringType()),
                            F.col('elevation_ft').cast(IntegerType()),
                            F.col('continent').cast(StringType()),
                            F.col('iso_country').cast(StringType()),
                            F.col('iso_region').cast(StringType()),
                            F.col('municipality').cast(StringType()),
                            F.col('gps_code').cast(StringType()),
                            F.col('iata_code').cast(StringType()),
                            F.col('local_code').cast(StringType()),
                            F.col('coordinates').cast(StringType())
                           )

city = city_df.select(
                      F.col('City').cast(StringType()).alias('city'),
                      F.col('State').cast(StringType()).alias('state'),
                      F.col('Median Age').cast(IntegerType()).alias('median_age'),
                      F.col('Male Population').cast(IntegerType()).alias('male_population'),
                      F.col('Female Population').cast(IntegerType()).alias('female_population'),
                      F.col('Total Population').cast(IntegerType()).alias('total_population'),
                      F.col('Number of Veterans').cast(IntegerType()).alias('number_of_veterans'),
                      F.col('Foreign-born').cast(IntegerType()).alias('foreign_born'),
                      F.col('Average Household Size').cast(FloatType()).alias('average_household_size'),
                      F.col('State Code').cast(StringType()).alias('state_code'),
                      F.col('Race').cast(StringType()).alias('race'),
                      F.col('Count').cast(IntegerType()).alias('count')
                     )

immigration.write.mode('overwrite').parquet(output_path+'immigration')
airport.write.mode('overwrite').parquet(output_path+'airport')
city.write.mode('overwrite').parquet(output_path+'city')

#### Data Quality 

We will check to see that the tables are correctly created, that they are correctly written to the output path and that there are no duplicates

In [114]:
print('immigration data ok:',immigration_df.count() == spark.read.parquet(output_path + 'immigration').count())
print('airport data ok:', airport_df.count() == spark.read.parquet(output_path + 'airport').count())
print('city data ok:', city_df.count() == spark.read.parquet(output_path + 'city').count())

immigration data ok: True
airport data ok: True
city data ok: True


In [117]:
print('No immigration duplicates:', (immigration.count()-immigration.distinct().count())==0)
print('No airport duplicates:', (airport.count()-airport.distinct().count())==0)
print('No city duplicates:', (city.count()-city.distinct().count())==0)

No immigration duplicates: True
No airport duplicates: True
No city duplicates: True


#### 5 - Write Up
* Pyspark was chosen for two main reasons, being open source it doesn't tie one to a vendor and it is very easy to develop pipelines with it given the easy DataFrame abstraction.
* The Data should be updated monthly it give time for analysts to develop reporst for the month and developers to manage possible errors in the new data.
* If the data were to be increased 100x more machines would be added to the cluster until the required processing speed was reached
* airflow would be used if scheduling jobs at specific times were needed, like for example at 7am every morning.
* if 100+ people needed to acces the data more machines would be added to the cluster, same as with the increase in data.